In [1]:
import os
import importlib.util

import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator

import numpy as np
import h5py
from uncertainties import unumpy, ufloat
import tensorflow as tf
import pickle

from alad_mod.alad import ALAD
from core.histogram_builder import *
from data.hlf_preprocessing import load

In [2]:
result_path = 'model/'
model_file = 'model-10000000'

print('loading alad')

# loading config
spec = importlib.util.spec_from_file_location('config', os.path.join(result_path, 'config.py'))
config_alad = importlib.util.module_from_spec(spec)
spec.loader.exec_module(config_alad)

# loading preprocessor
preprocessor = load(os.path.join(result_path, 'preprocessor.pkl'))

# loading alad
tf.reset_default_graph()
ad = ALAD(config_alad, tf.Session())
ad.load(os.path.join(result_path, model_file))

loading alad
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.batch_normalization` documentation).
Instructions for updating:
Use keras.layers.dropout instead.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in 

/home/oliverkn/pro/pycharm/pycharm/venv37/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.preprocessing.data module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.preprocessing. Anything that cannot be imported from sklearn.preprocessing is now part of the private API.
  warnings.warn(message, FutureWarning)
/home/oliverkn/pro/pycharm/pycharm/venv37/lib/python3.7/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RobustScaler from version 0.21.2 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/oliverkn/pro/pycharm/pycharm/venv37/lib/python3.7/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator OneHotEncoder from version 0.21.2 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at y

In [3]:
lum = 4429

W_proc_list = ['w1jets', 'w2jets', 'w3jets']
DY_proc_list = ['dy1jets', 'dy2jets', 'dy3jets', 'dy4jets']
ttbar_proc_list = ['ttbar']
data_proc_list = ['data']
b_proc_list = W_proc_list + DY_proc_list
bs_proc_list = b_proc_list + ttbar_proc_list
proc_list = bs_proc_list + data_proc_list

sets = {}
sets['dy1jets'] = {'file': '/home/oliverkn/pro/opendata_v2/7719/data_pre.hdf5', 'xsec':561, 'K':1.23}
sets['dy2jets'] = {'file': '/home/oliverkn/pro/opendata_v2/7721/data_pre.hdf5', 'xsec':181, 'K':1.23}
sets['dy3jets'] = {'file': '/home/oliverkn/pro/opendata_v2/7722/data_pre.hdf5', 'xsec':51, 'K':1.23}
sets['dy4jets'] = {'file': '/home/oliverkn/pro/opendata_v2/7723/data_pre.hdf5', 'xsec':15, 'K':1.23}

sets['w1jets'] = {'file': '/home/oliverkn/pro/opendata_v2/9863/data_pre.hdf5', 'xsec':4480, 'K':1.23}
sets['w2jets'] = {'file': '/home/oliverkn/pro/opendata_v2/9864/data_pre.hdf5', 'xsec':1435, 'K':1.23}
sets['w3jets'] = {'file': '/home/oliverkn/pro/opendata_v2/9865/data_pre.hdf5', 'xsec':304, 'K':1.23}

sets['ttbar'] = {'file': '/home/oliverkn/pro/opendata_v2/9588/data_pre.hdf5', 'xsec':164, 'K':1.66}
sets['data'] = {'file': '/home/oliverkn/pro/opendata_v2/6021/data_pre.hdf5', 'xsec':1, 'K':1}

In [4]:
data_dict = {}

def pre_select(x):
    filter_iso = x[:,7] + x[:,8] + x[:,9] < 0.1
    filter_eta = np.abs(x[:,5]) < 1.4
    filter_njets = x[:,2] > 1
    filter_idx = filter_iso * filter_eta * filter_njets
    
    return x[filter_idx]

for key, set in sets.items():
    print('-----------------loading %s-----------------' % key)

    file = set['file']
    hdf5_file = h5py.File(file, 'r')
    
    data = hdf5_file['data']
    n_tot = hdf5_file['n_tot'][()]
    n_tup = data.shape[0]
    
    if key == 'data':
        x = data[()]
    else:
        n_tot_target = lum * set['xsec'] * set['K']
        n_tup_target = int(n_tot_target/n_tot * n_tup)
        
        print('n_tup        = %d' % n_tup)
        print('n_tup_target = %d' % n_tup_target)
        if(n_tup < n_tup_target):
            print('***********WARNING***********: not enough samples available')
            
        x = data[0:n_tup_target]
    
    # run preselection
    # x = pre_select(x)
    
    sets[key]['x_pre'] = x

print('done')

-----------------loading dy1jets-----------------
n_tup        = 418047
n_tup_target = 403790
-----------------loading dy2jets-----------------
n_tup        = 154836
n_tup_target = 148215
-----------------loading dy3jets-----------------
n_tup        = 333484
n_tup_target = 44099
-----------------loading dy4jets-----------------
n_tup        = 267980
n_tup_target = 13323
-----------------loading w1jets-----------------
n_tup        = 2053589
n_tup_target = 1682703
-----------------loading w2jets-----------------
n_tup        = 715775
n_tup_target = 705553
-----------------loading w3jets-----------------
n_tup        = 171661
n_tup_target = 167878
-----------------loading ttbar-----------------
n_tup        = 2552906
n_tup_target = 128462
-----------------loading data-----------------
done


In [5]:
for key, set in sets.items():
    print('%s: %s' % (key, set['x_pre'].shape))

dy1jets: (403790, 23)
dy2jets: (148215, 23)
dy3jets: (44099, 23)
dy4jets: (13323, 23)
w1jets: (1682703, 23)
w2jets: (705553, 23)
w3jets: (167878, 23)
ttbar: (128462, 23)
data: (7711955, 23)


In [6]:
cont_bins = 25
hist_settings = {}
hist_settings['HT'] = {'symbol': 'Jets-$p_T$ scalar sum $H_T$ [GeV]', 'range': (0, 2000), 'yscale': 'log', 'bins': cont_bins, 'int': False}
hist_settings['mass_jet'] = {'symbol': 'Jets Mass $M_J$ [GeV]', 'range': (0, 3000), 'yscale': 'log', 'bins': cont_bins, 'int': False}
hist_settings['n_jet'] = {'symbol': 'Number of jets $N_J$', 'range': (0, 15), 'yscale': 'log', 'int': True, 'bin_size': 1}
hist_settings['n_bjet'] = {'symbol': 'Number of b-tagged jets $N_b$', 'range': (0, 6), 'yscale': 'log', 'int': True, 'bin_size': 1}
hist_settings['lep_pt'] = {'range': (20, 1000), 'yscale': 'log', 'bins': cont_bins, 'int': False}
hist_settings['lep_eta'] = {'range': (-2.5, 2.5), 'yscale': 'linear', 'bins': cont_bins, 'int': False}
hist_settings['lep_charge'] = {'range': (-1, 1), 'yscale': 'linear', 'int': True}
hist_settings['lep_iso_ch'] = {'range': (0, 0.1), 'yscale': 'log', 'bins': cont_bins, 'int': False}
hist_settings['lep_iso_neu'] = {'range': (0, 0.1), 'yscale': 'log', 'bins': cont_bins, 'int': False}
hist_settings['lep_iso_gamma'] = {'range': (0, 0.1), 'yscale': 'log', 'bins': cont_bins, 'int': False}
hist_settings['MET'] = {'range': (0, 1000), 'yscale': 'log', 'bins': cont_bins, 'int': False}
hist_settings['METo'] = {'range': (-100, 100), 'yscale': 'linear', 'bins': cont_bins, 'int': False}
hist_settings['METp'] = {'range': (-100, 100), 'yscale': 'linear', 'bins': cont_bins, 'int': False}
hist_settings['MT'] = {'range': (0, 200), 'yscale': 'log', 'bins': cont_bins, 'int': False}
hist_settings['n_mu'] = {'range': (0, 15), 'yscale': 'linear', 'int': True, 'bin_size': 1}
hist_settings['pt_mu'] = {'range': (0, 1000), 'yscale': 'log', 'bins': cont_bins, 'int': False}
hist_settings['mass_mu'] = {'range': (0, 1000), 'yscale': 'log', 'bins': cont_bins, 'int': False}
hist_settings['n_ele'] = {'range': (0, 15), 'yscale': 'linear', 'int': True, 'bin_size': 1}
hist_settings['pt_ele'] = {'range': (0, 1000), 'yscale': 'log', 'bins': cont_bins, 'int': False}
hist_settings['mass_ele'] = {'range': (0, 1000), 'yscale': 'log', 'bins': cont_bins, 'int': False}
hist_settings['n_neu'] = {'range': (0, 400), 'yscale': 'linear', 'int': True, 'bin_size': 1}
hist_settings['n_ch'] = {'range': (0, 1000), 'yscale': 'linear', 'int': True, 'bin_size': 1}
hist_settings['n_photon'] = {'range': (0, 1000), 'yscale': 'linear', 'int': True, 'bin_size': 1}

def build_hists(x, selection_functions):
    hist_builder_arr = [HistogramBuilder(hist_settings) for f in selection_functions]
    
    x_f = x
    for i, f in enumerate(selection_functions):
        x_f = f(x_f)
        hist_builder_arr[i].add_data(x_f)
      
    return [builder.get_histogram_data() for builder in hist_builder_arr]

hist_cut_proc_dict = {}

def build_all_hists(cut_functions, cut_names, proc_list=None):
    if proc_list is None:
        proc_list = sets.keys()
    
    for proc in proc_list:
        set = sets[proc]
        print('building hist for %s' % proc)
    
        # build hists
        hists = build_hists(set['x_pre'], cut_functions)
        
        # add hists to dict
        cut_name = 'pre'
        for i in range(len(hists)):
            cut_name += '-' + cut_names[i]
            
            if cut_name not in hist_cut_proc_dict:
                hist_cut_proc_dict[cut_name] = {}
                
            hist_cut_proc_dict[cut_name][proc] = hists[i]
            
def build_anomaly_score_select(thres_target):
    def anomaly_score_select(x, thres=thres_target, score_type='l1'):
        x_transformed = preprocessor.transform(x)
        scores = ad.get_anomaly_scores(x_transformed, type=score_type)  
        anomaly_idx = scores > thres
        return x[anomaly_idx]
    return anomaly_score_select

def build_anomaly_q_select(q_target):
    def anomaly_q_select(x, q=q_target, score_type='l1'):
        x_transformed = preprocessor.transform(x)
        scores = ad.get_anomaly_scores(x_transformed, type=score_type)  
        thres = np.quantile(scores, 1 - q)
        anomaly_idx = scores > thres
        return x[anomaly_idx]
    return anomaly_q_select

In [7]:
print('----------------------------building pre----------------------------')

def id_select(x):
    return x

build_all_hists([id_select], [''])
print('done')

----------------------------building pre----------------------------
building hist for dy1jets
building hist for dy2jets
building hist for dy3jets
building hist for dy4jets
building hist for w1jets
building hist for w2jets
building hist for w3jets
building hist for ttbar
building hist for data
done


In [8]:
print('----------------------------building anoscore----------------------------')

thres = 20
build_all_hists([build_anomaly_score_select(thres)], ['anoscore'])
print('done')

----------------------------building anoscore----------------------------
building hist for dy1jets
building hist for dy2jets
building hist for dy3jets
building hist for dy4jets
building hist for w1jets
building hist for w2jets
building hist for w3jets
building hist for ttbar
building hist for data
done


In [9]:
sum_n_pre = 0
sum_n_ano = 0
for proc in bs_proc_list:
    sum_n_pre += hist_cut_proc_dict['pre-'][proc]['HT'].n
    sum_n_ano += hist_cut_proc_dict['pre-anoscore'][proc]['HT'].n
    
for proc in bs_proc_list:
    n = hist_cut_proc_dict['pre-'][proc]['HT'].n
    print('%s: %d (%f)'%(proc, n, n/sum_n_pre))

print('--------------------------------------')
print('n_tup (mc):   %d' % sum_n_pre)
print('n_tup (data): %d' % hist_cut_proc_dict['pre-']['data']['HT'].n)

w1jets: 1682703 (0.510835)
w2jets: 705553 (0.214192)
w3jets: 167878 (0.050964)
dy1jets: 403790 (0.122583)
dy2jets: 148215 (0.044995)
dy3jets: 44099 (0.013388)
dy4jets: 13323 (0.004045)
ttbar: 128462 (0.038999)
--------------------------------------
n_tup (mc):   3294023
n_tup (data): 7711955


In [10]:
print('----------------------------building ano_q_b----------------------------')

target_q = 1000e-6

# compute threshold for b-processes to achive target_q
b_scores = np.empty(0)
for proc in b_proc_list:
    x = sets[proc]['x_pre']
    x_transformed = preprocessor.transform(x)
    scores = ad.get_anomaly_scores(x_transformed, type='l1')
    b_scores = np.concatenate([b_scores, scores])
thres = np.quantile(b_scores, 1 - target_q)
print('target thres = %d' % thres)

build_all_hists([build_anomaly_score_select(thres)], ['ano_q_b'], proc_list=b_proc_list)
build_all_hists([build_anomaly_q_select(target_q)], ['ano_q_b'], proc_list=['data'])
print('done')

----------------------------building ano_q_b----------------------------
target thres = 17
building hist for w1jets
building hist for w2jets
building hist for w3jets
building hist for dy1jets
building hist for dy2jets
building hist for dy3jets
building hist for dy4jets
building hist for data
done


In [11]:
print('----------------------------building ano_q_bs----------------------------')

target_q = 1000e-6

def post_select(x):
    filter_bjets =  x[:,3] > 1
    filter_njets = x[:,2] > 5
    filter_idx = filter_bjets * filter_njets
    
    return x[filter_idx]

# compute threshold for b-processes to achive target_q
bs_scores = np.empty(0)
for proc in bs_proc_list:
    x = sets[proc]['x_pre']
    x_transformed = preprocessor.transform(x)
    scores = ad.get_anomaly_scores(x_transformed, type='l1')
    bs_scores = np.concatenate([bs_scores, scores])
thres = np.quantile(bs_scores, 1 - target_q)
print('target thres = %d' % thres)

build_all_hists([build_anomaly_score_select(thres), post_select], ['ano_q_bs', 'pos'], proc_list=bs_proc_list)
build_all_hists([build_anomaly_q_select(target_q), post_select], ['ano_q_bs', 'pos'], proc_list=['data'])
print('done')

----------------------------building ano_q_bs----------------------------
target thres = 21
building hist for w1jets
building hist for w2jets
building hist for w3jets
building hist for dy1jets
building hist for dy2jets
building hist for dy3jets
building hist for dy4jets
building hist for ttbar
building hist for data
done


In [12]:
for cut, hist_proc_dict in hist_cut_proc_dict.items():  
    # sum W
    hist_W = sum_hists([hist_proc_dict[proc] for proc in W_proc_list])
    hist_proc_dict['W'] = hist_W
    
    # sum DY
    hist_DY = sum_hists([hist_proc_dict[proc] for proc in DY_proc_list])
    hist_proc_dict['DY'] = hist_DY
    
    #sum background
    hist_b = sum_hists([hist_proc_dict[proc] for proc in b_proc_list])
    hist_proc_dict['b'] = hist_b
    
    if cut == 'pre-ano_q_b':
        continue
        
    #sum background + signal
    hist_bs = sum_hists([hist_proc_dict[proc] for proc in bs_proc_list])
    hist_proc_dict['bs'] = hist_bs

# print event numbers
for cut, hist_proc_dict in hist_cut_proc_dict.items():  
    print('---------------------%s---------------------' % cut)
    for proc in ['W', 'DY', 'ttbar', 'b', 'bs', 'data']:
        if proc not in hist_proc_dict:
            continue
        
        hist_proc = hist_proc_dict[proc]
        n = hist_proc['HT'].n
        # print('%s total events:\t %d' %(proc, n))
        print('{:6} events: {} ({:6d}ppm)'.format(proc, n, int(n/hist_cut_proc_dict['pre-'][proc]['HT'].n*1e6)))

---------------------pre----------------------
W      events: 2556134 (1000000ppm)
DY     events: 609427 (1000000ppm)
ttbar  events: 128462 (1000000ppm)
b      events: 3165561 (1000000ppm)
bs     events: 3294023 (1000000ppm)
data   events: 7711955 (1000000ppm)
---------------------pre-anoscore---------------------
W      events: 1405 (   549ppm)
DY     events: 713 (  1169ppm)
ttbar  events: 1799 ( 14004ppm)
b      events: 2118 (   669ppm)
bs     events: 3917 (  1189ppm)
data   events: 7368 (   955ppm)
---------------------pre-ano_q_b---------------------
W      events: 2098 (   820ppm)
DY     events: 1068 (  1752ppm)
b      events: 3166 (  1000ppm)
data   events: 7712 (  1000ppm)
---------------------pre-ano_q_bs---------------------
W      events: 1185 (   463ppm)
DY     events: 600 (   984ppm)
ttbar  events: 1510 ( 11754ppm)
b      events: 1785 (   563ppm)
bs     events: 3295 (  1000ppm)
data   events: 7712 (  1000ppm)
---------------------pre-ano_q_bs-pos---------------------
W     

In [13]:
pickle.dump(hist_cut_proc_dict, open('output/plot_data.pkl', 'wb'))
pickle.dump(hist_settings, open('output/hist_settings.pkl', 'wb'))